In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#!pip install ray[train]

In [3]:
#!pip install -U ipywidgets
# not needed for production, only for jupyter notebook

In [4]:
#--- need to run 
#memcached -m 10240 -I 20m -vv -u www-data &

In [ ]:
#--- should clean imports - copy paste
import ray
from ray.train import ScalingConfig
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor, Normalize, Compose
import ray.train.torch
import ray.util.state
import ray
import torch
import torch.nn as nn
import ray.train.torch
import ray.train
from torch.nn import CrossEntropyLoss, MSELoss
from torch.optim import Adam
from quickannotator.constants import TileStatus

In [6]:
ray.init()

2025-01-22 01:10:25,337	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.12
Ray version:,2.37.0
Dashboard:,http://127.0.0.1:8265


In [ ]:
from quickannotator.dl.training import train_model

@ray.remote
class DLActor:
    def __init__(self,classid,status=-1):
        self.classid=classid
        self.status=status
        
    def start_dlproc(self,is_train=True,allow_pred=True, tile_size=2_048):
        #need to check if is already training, if yes, no opt
        print("starting up")
        
        
        
        #scaling_config = ray.train.ScalingConfig(num_workers=2, use_gpu=True,resources_per_worker={"GPU":.5})
        ## will fail on single node single gpu --- (DLActor pid=174521) Duplicate GPU detected : rank 1 and rank 0 both on CUDA device 1000

        #--- NEED A GPU CONTAINER TO TEST THIS        
        total_gpus  = ray.cluster_resources().get("GPU", 0)
        total_gpus_available=  ray.available_resources().get("GPU", 0)
        print(total_gpus,total_gpus_available)
        resource_to_request = (total_gpus_available/total_gpus)/2 * 1.01
        print(f"resource_to_request: {resource_to_request}")

        scaling_config = ray.train.ScalingConfig(num_workers=int(total_gpus), use_gpu=True, resources_per_worker={"GPU":resource_to_request})
        

        #----
#        scaling_config = ray.train.ScalingConfig(num_workers=1, use_gpu=False)
        trainer = ray.train.torch.TorchTrainer(train_model,
                                       scaling_config=scaling_config,
                                       train_loop_config={'is_train':is_train, 
                                                          'allow_pred':allow_pred,
                                                         'classid':self.classid,
                                                         'tile_size':tile_size})
        self.status = trainer.fit().hex() ##widly -- this doesn't save the result in the actor
        return self.status
    
    def infer(self,image_id,tileids,status=TileStatus.PROCESSING.value): #only bulk should be performed, there is no difference between doing 1 versus 100 tiles, so this reduces code complexity
        import sqlalchemy, datetime
        from quickannotator.dl.database import create_db_engine, get_database_path, get_session_aj
        from quickannotator.db import db_session, Project, Image, AnnotationClass, Notification, Tile, Setting, Annotation, SearchCache

        #open question --- seems like we're getting a lot of seessions, should be a factory process somewhere which also has a pool of existing sessions for reuse


        # probably need to *Delete* the tiles that are associated with this tileid before executing the below, so that they're not duplicated
        # i would do that in a seperate function as "infer" doesn't logically mean "delete", so the expected behavior might be weird if it delets stuff unprompted
        # perhaps that could be clarified with a e.g., named function paramter - or perhaps a seperate function is really needed
        session = get_session_aj(create_db_engine(get_database_path()))

                #id should be modified one we have https://github.com/choosehappy/QuickAnnotator/pull/58
                #imageid will need to be included - end result is a multikey -(imageid, tileid, classid)
        stmt = sqlalchemy.update(Tile).where(Tile.id.in_(tileids), Tile.image_id == image_id,
                                              Tile.annotation_class_id == self.classid)\
                                                .values(seen=status) #  ,date=datetime.datetime.now())) ## should add date time

        # Execute the update
        session.execute(stmt)
        session.commit()
        
        # Close the session
        session.close()


        #need a similar statement to get the DL starting
        return True
    

    def getTileStatus(self,tileids): #probably belongs else where but need this for debug
        import sqlalchemy, datetime
        from quickannotator.dl.database import create_db_engine, get_database_path, get_session_aj
        from quickannotator.db import db_session, Project, Image, AnnotationClass, Notification, Tile, Setting, Annotation, SearchCache

        
        session = get_session_aj(create_db_engine(get_database_path()))

        stmt = session.query(Tile).filter(Tile.id.in_(tileids),
                          Tile.annotation_class_id == self.classid)

        # Execute the query
        result = stmt.all()
        
        # Close the session
        session.close()

        return result


    def getclassid(self):
        return self.classid
        
    
    def getstatus(self):
        print("in get status!")
        print(f"self.status:\t {self.status}")
        return self.status

    def setstatus(self,status):
        self.status=status
        return self.status



ModuleNotFoundError: No module named 'quickannotator'

this section  is some non-ray testing to see if things work as expected. to use these, remove the @ray annotator in the actor above 

In [6]:
##not ray test
class2=DLActor(classid=2)


In [ ]:
class2.infer(image_id=1,tileids=[323],status=1)

In [ ]:
# -- check it was set
tile=class2.getTileStatus([323])
tile[0].seen

In [ ]:
ref2=class2.start_dlproc()


This section is when the @Ray actor is enabled, and thus has remote. note we need max_concurrency = 2 for this to work

In [14]:
class2 = DLActor.options(max_concurrency=2).remote(classid=2)

In [15]:
print(ray.get(class2.getclassid.remote()))


(pid=33913) InitSpatiaMetaData() error:"table spatial_ref_sys already exists"
(DLActor pid=33913) InitSpatiaMetaData() error:"table spatial_ref_sys already exists"


2


In [16]:
print(ray.get(class2.infer.remote(image_id=1,tileids=[323],status=1)))

True


In [17]:
tile=(ray.get(class2.getTileStatus.remote([323])))
print(tile[0].seen)


1


In [18]:
ref2=class2.start_dlproc.remote()
print(ref2)

ObjectRef(80e22aed7718a125e07aa470902f18f1f27993e90100000001000000)


2025-01-21 23:42:04,767	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray.exceptions.RayTaskError(ValueError): ray::_Inner.train() (pid=33985, ip=172.17.0.2, actor_id=f40a15eae28830e4752362e701000000, repr=TorchTrainer)
  File "/opt/venv/lib/python3.10/site-packages/ray/air/_internal/util.py", line 98, in run
    self._ret = self._target(*self._args, **self._kwargs)
  File "/opt/venv/lib/python3.10/site-packages/ray/tune/trainable/function_trainable.py", line 45, in <lambda>
    training_func=lambda: self._trainable_func(self.config),
  File "/opt/venv/lib/python3.10/site-packages/ray/train/base_trainer.py", line 799, in _trainable_func
    super()._trainable_func(self._merged_config)
  File "/opt/venv/lib/python3.10/site-packages/ray/tune/trainable/function_trainable.py", line 248, in _trainable_func
    output = fn()
  File "/opt/venv/lib/python3.10/site-packages/ray/train/base_trainer.py", line 107, in _train_coordinator_fn
    trainer.train